Modifying LLM-as-a-Judge system as a majority voting system consisting of 3 LLMs to counteract self-enhancement bias.

First, evaluate the judge based on gpt-5 responses.

Then give evaluation results for sonnet-4.5 and gemini-3.


In [ ]:
!pip install --upgrade gspread gspread_dataframe
!pip install anthropic
import os
from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata
GEMINI_API_KEY=userdata.get('LLM_JUDGE')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
SONNET_API_KEY = userdata.get('SONNET_API_KEY')
import google.generativeai as genai
from google.genai.types import ThinkingConfig, ThinkingLevel
from PIL import Image
import json
from google import genai
from google.genai import types
import time
import pickle
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default
from gspread_dataframe import get_as_dataframe
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
import base64
import mimetypes
from typing import List, Optional
from anthropic import Anthropic
from openai import OpenAI
import asyncio
import time
import base64
import mimetypes
import json
from anthropic import AsyncAnthropic
from openai import AsyncOpenAI
from google import genai
from google.genai import types

LLMs Responses

In [3]:
chapters_keys_gpt_and_claude = {
  '2':'Chapter_2_Vectors_4',
  '5':'Chapter_5_Newton_laws_of_motion_12',
  '6':'Chapter_6_Applications_of_Newton_Laws_18',
  '7':'Chapter_7_Work_and_Kinetic_Energy_5',
  '8':'Chapter_8_Potential_Energy_and_Conservation_of_energy_8',
  '9':'Chapter_9_Linear_Momentum_and_Collisions_11',
  '10':'Chapter_10_Fixed_Axis_Rotation_15',
  '11':'Chapter_11_Angular_Momentum_6',
  '12':'Chapter_12_Static_Equilibrium_and_Elasticity_13',
  '14':'Chapter_14_Fluid_Mechanics_5',
  '17':'Chapter_17_Sound_7'}

chapters_keys_gemini = {
  '2':'Chapter_2_Vectors_2',
  '5':'Chapter_5_Newton_laws_of_motion_4',
  '6':'Chapter_6_Applications_of_Newton_Laws_10',
  '7':'Chapter_7_Work_and_Kinetic_Energy_1',
  '8':'Chapter_8_Potential_Energy_and_Conservation_of_energy_3',
  '9':'Chapter_9_Linear_Momentum_and_Collisions_4',
  '10':'Chapter_10_Fixed_Axis_Rotation_6',
  '11':'Chapter_11_Angular_Momentum_2',
  '12':'Chapter_12_Static_Equilibrium_and_Elasticity_10',
  '14':'Chapter_14_Fluid_Mechanics_1',
  '17':'Chapter_17_Sound_2'}


gpt_5_path = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/GPT_answers_diagram_based_questions.json"
claude_path = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/anthropic_answers_diagram_based_questions.json"
gemini_path = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/gemini_answers_intermediate.json"

with open(gpt_5_path, "r", encoding="utf-8") as f:
  gpt5 = json.load(f)

with open(claude_path, "r", encoding="utf-8") as f:
  claude = json.load(f)

with open(gemini_path, "r", encoding="utf-8") as f:
  gemini_responses = json.load(f)

In [4]:
# Loading groundtruth labels
output_directory = "/content/drive/MyDrive/Medhavi AI/Physics QA Datasets"
output_filename = "lbled_output.pkl"
output_filepath = f"{output_directory}/{output_filename}"

with open(output_filepath, 'rb') as f:
    loaded_lbled_output = pickle.load(f)

total_questions = 0
for k in loaded_lbled_output.keys():
  total_questions += len(loaded_lbled_output[k])


print(f"Successfully loaded object from '{output_filepath}'")
print(f"Keys of loaded object: {loaded_lbled_output.keys()}")
print(f"Total questions:  {total_questions}")

Successfully loaded object from '/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/lbled_output.pkl'
Keys of loaded object: dict_keys(['2', '5', '6', '7', '8', '9', '10', '11', '12', '14', '17'])
Total questions:  45


In [46]:
def LLM_Judge(model_response, image_path):
    gemini_client = genai.Client(api_key=GEMINI_API_KEY)

    with open(image_path, "rb") as f:
        image_bytes = f.read()

    system_prompt = """
You are an answer verification system.

Your task is to compare the ground truth answer extracted from the image
with the model answer provided in the JSON and return a binary verdict.

Core Instructions:
1. Extract the ground truth answer exactly as shown in the image.
2. Read the model’s answer from the provided JSON.
3. Compare all subparts (if the question has multiple parts).
4. If even one subpart is incorrect, incomplete, or missing, the verdict MUST be Wrong.
5. Do not infer intent or give partial credit.

Semantic Equivalence Requirement (MANDATORY):
Before deciding Correct or Wrong, you MUST check whether the ground truth
and model answer are semantically equivalent even if expressed in different valid forms.
You MUST normalize both answers into a common canonical representation before comparison.

Acceptable equivalence transformations include (when applicable):
- Vector representations: Cartesian components (ai + bj), magnitude–direction form,
  unit-vector form, or polar vs Cartesian. Convert one representation into the other
  and compare consistently.
- Reference frame differences: If each answer explicitly states a different origin or
  reference point, transform coordinates into the same reference frame before comparison.
- Algebraic equivalence: Simplified vs unsimplified expressions, factored vs expanded forms,
  exact vs approximate values (e.g., sqrt(2) vs 1.414).
- Derived equivalence: If one answer gives magnitude and direction, compute the implied
  components (or vice versa) and compare.

Numeric Tolerance Rule (apply AFTER canonicalization):
- Minor numerical differences caused by rounding or approximation MUST be treated as Correct.
- Use a reasonable tolerance (approximately 1% relative difference or small absolute error),
  unless the problem explicitly requires exact precision.
- Examples of acceptable matches: 5 vs 4.99, 3.28 vs 3.29, 25 vs 24.85.

Disallowed Equivalence:
- Do NOT mark Correct if equivalence would require changing physical assumptions,
  ignoring stated reference frames, flipping axes without justification,
  or inventing unstated transformations.

Output Rules:
1. Output exactly one line.
2. Use the following format verbatim:
   verdict: <Correct|Wrong>, ground_truth: <value>, model_answer: <value>
3. If there are multiple subparts, list them as comma-separated pairs.
4. Only output the final line. Do not include explanations, reasoning, or extra text.

Example:
verdict: Correct, ground_truth: a=3, b=7, model_answer: a=2.98, b=6.99
"""
    gemini_start = time.perf_counter()
    gemini_response = gemini_client.models.generate_content(
        model="gemini-3-pro-preview",
        contents=[
            types.Part(text=model_response),
            types.Part.from_bytes(data=image_bytes, mime_type="image/png")
        ],
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            thinking_config=types.ThinkingConfig(thinking_level="HIGH"),
            max_output_tokens=7000
        )
    )

    gemini_response_text = gemini_response.text
    gemini_elapsed_s = time.perf_counter() - gemini_start

    gpt_start = time.perf_counter()
    openAIClient = OpenAI(api_key=OPENAI_API_KEY)
    mime, _ = mimetypes.guess_type(image_path)
    mime = mime or "image/png"
    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    data_url = f"data:{mime};base64,{b64}"


    gpt_response = openAIClient.responses.create(
        model="gpt-5",
        input=[
            {
            "role": "system",
            "content": [
                {"type": "input_text", "text": system_prompt}
            ]
            },
            {
            "role": "user",
            "content": [
                {"type": "input_text",
                "text": model_response},
                {"type": "input_image",
                "image_url": data_url}
                ]
            }],
            reasoning={"effort": "high"},
            text={"verbosity": "low"},
    )
    gpt_response_text = gpt_response.output_text
    gpt_elapsed_s = time.perf_counter() - gpt_start

    sonnet_start = time.perf_counter()
    anthropicClient = Anthropic(api_key=SONNET_API_KEY)
    diagram_path = image_path
    CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
    msg_img = anthropicClient.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=7000,
        system=system_prompt,
        thinking={"type": "enabled", "budget_tokens": 6000},
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {"type": "base64", "media_type": mime, "data": b64},
                    },
                    {"type": "text", "text": json.dumps(model_response)},
                ],
            }
        ],
    )

    sonnet_response_text = "".join(
        block.text for block in msg_img.content if getattr(block, "type", None) == "text"
    )
    sonnet_elapsed_s = time.perf_counter() - sonnet_start

    return f"""

    Sonnet-4.5 Judge: {sonnet_response_text}
    Sonnet-4.5 Judge Time: {sonnet_elapsed_s}

    Gemini-3 Judge: {gemini_response_text}
    Gemini-3 Judge Time: {gemini_elapsed_s}

    GPT-5 Judge: {gpt_response_text}
    GPT-5 Judge Time: {gpt_elapsed_s}

    """




In [50]:
# GPT Judge Loop
checkpoint_path = '/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/gpt_judge_results.json'

# Load existing results if file exists, otherwise start fresh
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'r') as f:
        judge_results = json.load(f)
    print(f"Loaded existing checkpoint with {sum(len(v) for v in judge_results.values())} entries")
else:
    judge_results = {}
    print("Starting fresh")

# Judge Loop
base_path = '/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/Diagram_Essential_Questions/Correct_Freq/'

try:
    for folder_name in sorted(os.listdir(base_path)):
        chapter_path = base_path + folder_name
        chapter_number = folder_name.split('_')[1]

        if chapter_number not in judge_results:
            judge_results[chapter_number] = {}

        for question_number in sorted(os.listdir(chapter_path)):

            # Skip already completed questions
            if question_number in judge_results[chapter_number]:
                print(f"Skipping Chap #{chapter_number} Q #{question_number} (already done)")
                continue

            try:
                gpt5_response = gpt5[chapters_keys_gpt_and_claude[chapter_number]][question_number]
                sonnet_response = claude[chapters_keys_gpt_and_claude[chapter_number]][question_number]
                print(f"LABELED ANSWER: Chap #{chapter_number} Q #{question_number} {loaded_lbled_output[chapter_number][question_number]['gpt']}")

                ans_path = chapter_path + '/' + question_number + '/a.png'
                judge_response_gpt = LLM_Judge(gpt5_response, ans_path)
                print(f"LLM JUDGES RESPONSE: {judge_response_gpt}")

                # Save result
                judge_results[chapter_number][question_number] = judge_response_gpt

                # Write checkpoint to Drive after every question
                with open(checkpoint_path, 'w') as f:
                    json.dump(judge_results, f, indent=2)

            except Exception as e:
                print(f"\nERROR at Chap #{chapter_number} Q #{question_number}: {e}")
                print(f"Restart from: chapter={chapter_number}, question={question_number}")

                # Save whatever we have so far before continuing
                with open(checkpoint_path, 'w') as f:
                    json.dump(judge_results, f, indent=2)

                continue  # Skip this question and move on, or use `raise` to stop

except Exception as e:
    print(f"\nFATAL ERROR: {e}")
    print("Check checkpoint file for last successful entry")
    # Final save
    with open(checkpoint_path, 'w') as f:
        json.dump(judge_results, f, indent=2)
    raise

  # RESULT
  # Chap #17 Q #41 (Judge Disagreement) [Question's official answer key is wrong]
  # Chap #6 Q #51 (All judges wrong) [Question's official answer key is wrong]

Starting fresh
LABELED ANSWER: Chap #10 Q #115 Wrong
LLM JUDGES RESPONSE: 

    Sonnet-4.5 Judge: verdict: Wrong, ground_truth: ω=1.23 rad/s, model_answer: ω=1.34 rad/s
    Sonnet-4.5 Judge Time: 34.63546291400053

    Gemini-3 Judge: verdict: Wrong, ground_truth: 1.23 rad/s, model_answer: 1.34 rad/s
    Gemini-3 Judge Time: 38.610948862999976

    GPT-5 Judge: verdict: Wrong, ground_truth: 1.23 rad/s, model_answer: 1.34 rad/s
    GPT-5 Judge Time: 18.538622045999546

    
LABELED ANSWER: Chap #10 Q #123 Wrong
LLM JUDGES RESPONSE: 

    Sonnet-4.5 Judge: verdict: Wrong, ground_truth: I = (2/5)mR^2 = 0.4mR^2, model_answer: I = (3/5)mR^2 = 0.6mR^2
    Sonnet-4.5 Judge Time: 37.28516481099996

    Gemini-3 Judge: verdict: Wrong, ground_truth: I = 2/5 mR^2, model_answer: I = 0.60 m R^2
    Gemini-3 Judge Time: 44.74914020600045

    GPT-5 Judge: verdict: Wrong, ground_truth: I=2/5 m R^2, model_answer: I=0.60 m R^2
    GPT-5 Judge Time: 42.3813671580001

    
LABELED ANSWER: Chap #10 Q #65 

In [ ]:
# Judge System 100% fair for GPT-5 responses
# Evaluating sonnet-4.5 and gemini-3 responses
# Also, trying to optimize the Judge code by calling all 3 APIs in parallel

In [8]:
import asyncio
import time
import base64
import mimetypes
import json
from anthropic import AsyncAnthropic
from openai import AsyncOpenAI
from google import genai
from google.genai import types

async def Fast_LLM_Judge(model_response, image_path):
    system_prompt = """
You are an answer verification system.

Your task is to compare the ground truth answer extracted from the image
with the model answer provided in the JSON and return a binary verdict.

Core Instructions:
1. Extract the ground truth answer exactly as shown in the image.
2. Read the model's answer from the provided JSON.
3. Compare all subparts (if the question has multiple parts).
4. If even one subpart is incorrect, incomplete, or missing, the verdict MUST be Wrong.
5. Do not infer intent or give partial credit.

Semantic Equivalence Requirement (MANDATORY):
Before deciding Correct or Wrong, you MUST check whether the ground truth
and model answer are semantically equivalent even if expressed in different valid forms.
You MUST normalize both answers into a common canonical representation before comparison.

Acceptable equivalence transformations include (when applicable):
- Vector representations: Cartesian components (ai + bj), magnitude–direction form,
  unit-vector form, or polar vs Cartesian. Convert one representation into the other
  and compare consistently.
- Reference frame differences: If each answer explicitly states a different origin or
  reference point, transform coordinates into the same reference frame before comparison.
- Algebraic equivalence: Simplified vs unsimplified expressions, factored vs expanded forms,
  exact vs approximate values (e.g., sqrt(2) vs 1.414).
- Derived equivalence: If one answer gives magnitude and direction, compute the implied
  components (or vice versa) and compare.

Numeric Tolerance Rule (apply AFTER canonicalization):
- Minor numerical differences caused by rounding or approximation MUST be treated as Correct.
- Use a reasonable tolerance (approximately 1% relative difference or small absolute error),
  unless the problem explicitly requires exact precision.
- Examples of acceptable matches: 5 vs 4.99, 3.28 vs 3.29, 25 vs 24.85.

Disallowed Equivalence:
- Do NOT mark Correct if equivalence would require changing physical assumptions,
  ignoring stated reference frames, flipping axes without justification,
  or inventing unstated transformations.

Output Rules:
1. Output exactly one line.
2. Use the following format verbatim:
   verdict: <Correct|Wrong>, ground_truth: <value>, model_answer: <value>
3. If there are multiple subparts, list them as comma-separated pairs.
4. Only output the final line. Do not include explanations, reasoning, or extra text.

Example:
verdict: Correct, ground_truth: a=3, b=7, model_answer: a=2.98, b=6.99
"""

    # Prepare image data once, reuse across all calls
    mime, _ = mimetypes.guess_type(image_path)
    mime = mime or "image/png"
    with open(image_path, "rb") as f:
        image_bytes = f.read()
    b64 = base64.b64encode(image_bytes).decode("utf-8")
    data_url = f"data:{mime};base64,{b64}"
    model_response_json = json.dumps(model_response)

    # ── Individual async call functions ───────────────────────────────────────

    async def call_gemini():
        start = time.perf_counter()
        gemini_client = genai.Client(api_key=GEMINI_API_KEY)
        response = await asyncio.to_thread(
            gemini_client.models.generate_content,
            model="gemini-3-pro-preview",
            contents=[
                types.Part(text=model_response_json),
                types.Part.from_bytes(data=image_bytes, mime_type="image/png")
            ],
            config=types.GenerateContentConfig(
                system_instruction=system_prompt,
                thinking_config=types.ThinkingConfig(thinking_level="HIGH"),
                max_output_tokens=7000
            )
        )
        return response.text, time.perf_counter() - start

    async def call_sonnet():
        start = time.perf_counter()
        anthropicClient = AsyncAnthropic(api_key=SONNET_API_KEY)
        response = await anthropicClient.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=7000,
            system=system_prompt,
            thinking={"type": "enabled", "budget_tokens": 6000},
            messages=[{
                "role": "user",
                "content": [
                    {"type": "image", "source": {"type": "base64", "media_type": mime, "data": b64}},
                    {"type": "text", "text": model_response_json},
                ],
            }]
        )
        text = "".join(block.text for block in response.content if getattr(block, "type", None) == "text")
        return text, time.perf_counter() - start

    async def call_gpt():
        start = time.perf_counter()
        openAIClient = AsyncOpenAI(api_key=OPENAI_API_KEY)
        response = await openAIClient.responses.create(
            model="gpt-5",
            input=[
                {"role": "system", "content": [{"type": "input_text", "text": system_prompt}]},
                {"role": "user",   "content": [{"type": "input_text", "text": model_response_json},
                                               {"type": "input_image", "image_url": data_url}]}
            ],
            reasoning={"effort": "high"},
            text={"verbosity": "low"},
        )
        return response.output_text, time.perf_counter() - start

    # ── Fire all three simultaneously ─────────────────────────────────────────
    (gemini_text, gemini_elapsed), \
    (sonnet_text, sonnet_elapsed), \
    (gpt_text,    gpt_elapsed) = await asyncio.gather(
        call_gemini(),
        call_sonnet(),
        call_gpt(),
    )

    return f"""
    Sonnet-4.5 Judge: {sonnet_text}
    Sonnet-4.5 Judge Time: {sonnet_elapsed:.3f}s

    Gemini-3 Judge: {gemini_text}
    Gemini-3 Judge Time: {gemini_elapsed:.3f}s

    GPT-5 Judge: {gpt_text}
    GPT-5 Judge Time: {gpt_elapsed:.3f}s
    """

In [9]:
async def run_judge_loop():
    # ── Load or initialise each results dict ─────────────────────────────────
    checkpoint_path_sonnet = '/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/judge_results_sonnet.json'
    checkpoint_path_gemini = '/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/judge_results_gemini.json'

    if os.path.exists(checkpoint_path_sonnet):
        with open(checkpoint_path_sonnet, 'r') as f:
            judge_results_sonnet = json.load(f)
        print(f"[Sonnet] Loaded checkpoint with {sum(len(v) for v in judge_results_sonnet.values())} entries")
    else:
        judge_results_sonnet = {}
        print("[Sonnet] Starting fresh")

    if os.path.exists(checkpoint_path_gemini):
        with open(checkpoint_path_gemini, 'r') as f:
            judge_results_gemini = json.load(f)
        print(f"[Gemini] Loaded checkpoint with {sum(len(v) for v in judge_results_gemini.values())} entries")
    else:
        judge_results_gemini = {}
        print("[Gemini] Starting fresh")

    # ── Judge Loop ────────────────────────────────────────────────────────────
    base_path = '/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/Diagram_Essential_Questions/Correct_Freq/'

    try:
        for folder_name in sorted(os.listdir(base_path)):
            chapter_path   = base_path + folder_name
            chapter_number = folder_name.split('_')[1]

            if chapter_number not in judge_results_sonnet: judge_results_sonnet[chapter_number] = {}
            if chapter_number not in judge_results_gemini: judge_results_gemini[chapter_number] = {}

            for question_number in sorted(os.listdir(chapter_path)):

                ans_path = chapter_path + '/' + question_number + '/a.png'
                print(f"Chap #{chapter_number} Q #{question_number}")

                # ── Sonnet ────────────────────────────────────────────────────
                if question_number in judge_results_sonnet[chapter_number]:
                    print(f"  [Sonnet] Skipping (already done)")
                else:
                    try:
                        sonnet_response = claude[chapters_keys_gpt_and_claude[chapter_number]][question_number]
                        judge_results_sonnet[chapter_number][question_number] = await Fast_LLM_Judge(sonnet_response, ans_path)
                        print(f"  [Sonnet] {judge_results_sonnet[chapter_number][question_number]}")
                        with open(checkpoint_path_sonnet, 'w') as f:
                            json.dump(judge_results_sonnet, f, indent=2)
                    except Exception as e:
                        print(f"  [Sonnet] ERROR Chap #{chapter_number} Q #{question_number}: {e}")
                        print(f"  Restart from: chapter={chapter_number}, question={question_number}")
                        with open(checkpoint_path_sonnet, 'w') as f:
                            json.dump(judge_results_sonnet, f, indent=2)

                # ── Gemini ────────────────────────────────────────────────────
                if question_number in judge_results_gemini[chapter_number]:
                    print(f"  [Gemini] Skipping (already done)")
                else:
                    try:
                        gemini_response = gemini_responses[chapters_keys_gemini[chapter_number]][question_number]
                        judge_results_gemini[chapter_number][question_number] = await Fast_LLM_Judge(gemini_response, ans_path)
                        print(f"  [Gemini] {judge_results_gemini[chapter_number][question_number]}")
                        with open(checkpoint_path_gemini, 'w') as f:
                            json.dump(judge_results_gemini, f, indent=2)
                    except Exception as e:
                        print(f"  [Gemini] ERROR Chap #{chapter_number} Q #{question_number}: {e}")
                        print(f"  Restart from: chapter={chapter_number}, question={question_number}")
                        with open(checkpoint_path_gemini, 'w') as f:
                            json.dump(judge_results_gemini, f, indent=2)

    except Exception as e:
        print(f"\nFATAL ERROR: {e}")
        print("Check checkpoint files for last successful entries")
        with open(checkpoint_path_sonnet, 'w') as f: json.dump(judge_results_sonnet, f, indent=2)
        with open(checkpoint_path_gemini, 'w') as f: json.dump(judge_results_gemini, f, indent=2)
        raise

await run_judge_loop()

[Sonnet] Loaded checkpoint with 0 entries
[Gemini] Starting fresh
Chap #10 Q #115
  [Sonnet] 
    Sonnet-4.5 Judge: verdict: Wrong, ground_truth: ω=1.23 rad/s, model_answer: ω=1.40 rad/s
    Sonnet-4.5 Judge Time: 28.613s

    Gemini-3 Judge: verdict: Wrong, ground_truth: 1.23 rad/s, model_answer: 1.40 rad/s
    Gemini-3 Judge Time: 22.762s

    GPT-5 Judge: verdict: Wrong, ground_truth: ω = 1.23 rad/s, model_answer: ω = 1.40 rad/s
    GPT-5 Judge Time: 30.581s
    
  [Gemini] 
    Sonnet-4.5 Judge: verdict: Correct, ground_truth: ω=1.23 rad/s, model_answer: ω=1.22 rad/s
    Sonnet-4.5 Judge Time: 46.794s

    Gemini-3 Judge: verdict: Wrong, ground_truth: I = 8.8 kg·m^2, ω = 1.23 rad/s, model_answer: I = 9.33 kg·m^2, ω = 1.22 rad/s
    Gemini-3 Judge Time: 51.647s

    GPT-5 Judge: verdict: Correct, ground_truth: ω=1.23 rad/s, model_answer: 1.22 rad/s
    GPT-5 Judge Time: 45.222s
    
Chap #10 Q #123
  [Sonnet] 
    Sonnet-4.5 Judge: verdict: Correct, ground_truth: I=(2/5)mR^2, model_

In [10]:
async def Even_Faster_LLM_Judge(sonnet_response, gemini_response,image_path):
    system_prompt = """
You are an answer verification system.

Your task is to compare the ground truth answer extracted from the image
with the model answer provided in the JSON and return a binary verdict.

Core Instructions:
1. Extract the ground truth answer exactly as shown in the image.
2. Read the model's answer from the provided JSON.
3. Compare all subparts (if the question has multiple parts).
4. If even one subpart is incorrect, incomplete, or missing, the verdict MUST be Wrong.
5. Do not infer intent or give partial credit.

Semantic Equivalence Requirement (MANDATORY):
Before deciding Correct or Wrong, you MUST check whether the ground truth
and model answer are semantically equivalent even if expressed in different valid forms.
You MUST normalize both answers into a common canonical representation before comparison.

Acceptable equivalence transformations include (when applicable):
- Vector representations: Cartesian components (ai + bj), magnitude–direction form,
  unit-vector form, or polar vs Cartesian. Convert one representation into the other
  and compare consistently.
- Reference frame differences: If each answer explicitly states a different origin or
  reference point, transform coordinates into the same reference frame before comparison.
- Algebraic equivalence: Simplified vs unsimplified expressions, factored vs expanded forms,
  exact vs approximate values (e.g., sqrt(2) vs 1.414).
- Derived equivalence: If one answer gives magnitude and direction, compute the implied
  components (or vice versa) and compare.

Numeric Tolerance Rule (apply AFTER canonicalization):
- Minor numerical differences caused by rounding or approximation MUST be treated as Correct.
- Use a reasonable tolerance (approximately 1% relative difference or small absolute error),
  unless the problem explicitly requires exact precision.
- Examples of acceptable matches: 5 vs 4.99, 3.28 vs 3.29, 25 vs 24.85.

Disallowed Equivalence:
- Do NOT mark Correct if equivalence would require changing physical assumptions,
  ignoring stated reference frames, flipping axes without justification,
  or inventing unstated transformations.

Output Rules:
1. Output exactly one line.
2. Use the following format verbatim:
   verdict: <Correct|Wrong>, ground_truth: <value>, model_answer: <value>
3. If there are multiple subparts, list them as comma-separated pairs.
4. Only output the final line. Do not include explanations, reasoning, or extra text.

Example:
verdict: Correct, ground_truth: a=3, b=7, model_answer: a=2.98, b=6.99
"""

    # Prepare image data once, reuse across all calls
    mime, _ = mimetypes.guess_type(image_path)
    mime = mime or "image/png"
    with open(image_path, "rb") as f:
        image_bytes = f.read()
    b64 = base64.b64encode(image_bytes).decode("utf-8")
    data_url = f"data:{mime};base64,{b64}"
    sonnet_response_json = json.dumps(sonnet_response)
    gemini_response_json = json.dumps(gemini_response)

    # ── Individual async call functions for gemini response ───────────────────────────────────────

    async def call_gemini_gemini():
        start = time.perf_counter()
        gemini_client = genai.Client(api_key=GEMINI_API_KEY)
        response = await asyncio.to_thread(
            gemini_client.models.generate_content,
            model="gemini-3-pro-preview",
            contents=[
                types.Part(text=gemini_response_json),
                types.Part.from_bytes(data=image_bytes, mime_type="image/png")
            ],
            config=types.GenerateContentConfig(
                system_instruction=system_prompt,
                thinking_config=types.ThinkingConfig(thinking_level="HIGH"),
                max_output_tokens=7000
            )
        )
        return response.text, time.perf_counter() - start

    async def call_sonnet_gemini():
        start = time.perf_counter()
        anthropicClient = AsyncAnthropic(api_key=SONNET_API_KEY)
        response = await anthropicClient.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=7000,
            system=system_prompt,
            thinking={"type": "enabled", "budget_tokens": 6000},
            messages=[{
                "role": "user",
                "content": [
                    {"type": "image", "source": {"type": "base64", "media_type": mime, "data": b64}},
                    {"type": "text", "text": gemini_response_json},
                ],
            }]
        )
        text = "".join(block.text for block in response.content if getattr(block, "type", None) == "text")
        return text, time.perf_counter() - start

    async def call_gpt_gemini():
        start = time.perf_counter()
        openAIClient = AsyncOpenAI(api_key=OPENAI_API_KEY)
        response = await openAIClient.responses.create(
            model="gpt-5",
            input=[
                {"role": "system", "content": [{"type": "input_text", "text": system_prompt}]},
                {"role": "user",   "content": [{"type": "input_text", "text": gemini_response_json},
                                               {"type": "input_image", "image_url": data_url}]}
            ],
            reasoning={"effort": "high"},
            text={"verbosity": "low"},
        )
        return response.output_text, time.perf_counter() - start
    # ── Individual async call functions for gemini response ───────────────────────────────────────

    async def call_gemini_sonnet():
        start = time.perf_counter()
        gemini_client = genai.Client(api_key=GEMINI_API_KEY)
        response = await asyncio.to_thread(
            gemini_client.models.generate_content,
            model="gemini-3-pro-preview",
            contents=[
                types.Part(text=sonnet_response_json),
                types.Part.from_bytes(data=image_bytes, mime_type="image/png")
            ],
            config=types.GenerateContentConfig(
                system_instruction=system_prompt,
                thinking_config=types.ThinkingConfig(thinking_level="HIGH"),
                max_output_tokens=7000
            )
        )
        return response.text, time.perf_counter() - start

    async def call_sonnet_sonnet():
        start = time.perf_counter()
        anthropicClient = AsyncAnthropic(api_key=SONNET_API_KEY)
        response = await anthropicClient.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=7000,
            system=system_prompt,
            thinking={"type": "enabled", "budget_tokens": 6000},
            messages=[{
                "role": "user",
                "content": [
                    {"type": "image", "source": {"type": "base64", "media_type": mime, "data": b64}},
                    {"type": "text", "text": sonnet_response_json},
                ],
            }]
        )
        text = "".join(block.text for block in response.content if getattr(block, "type", None) == "text")
        return text, time.perf_counter() - start

    async def call_gpt_sonnet():
        start = time.perf_counter()
        openAIClient = AsyncOpenAI(api_key=OPENAI_API_KEY)
        response = await openAIClient.responses.create(
            model="gpt-5",
            input=[
                {"role": "system", "content": [{"type": "input_text", "text": system_prompt}]},
                {"role": "user",   "content": [{"type": "input_text", "text": sonnet_response_json},
                                               {"type": "input_image", "image_url": data_url}]}
            ],
            reasoning={"effort": "high"},
            text={"verbosity": "low"},
        )
        return response.output_text, time.perf_counter() - start

    # ── Fire all three simultaneously ─────────────────────────────────────────
    (gemini_judges_gemini, gemini_judges_gemini_time), \
    (sonnet_judges_gemini, sonnet_judges_gemini_time), \
    (gpt_judges_gemini, gpt_judges_gemini_time), \
    (gemini_judges_sonnet, gemini_judges_sonnet_time), \
    (sonnet_judges_sonnet, sonnet_judges_sonnet_time), \
    (gpt_judges_sonnet, gpt_judges_sonnet_time) = await asyncio.gather(
        call_gemini_gemini(),
        call_sonnet_gemini(),
        call_gpt_gemini(),
        call_gemini_sonnet(),
        call_sonnet_sonnet(),
        call_gpt_sonnet()
    )

    return f"""
    Sonnet-4.5 Judges Gemini: {sonnet_judges_gemini}
    Time: {sonnet_judges_gemini_time:.3f}s

    Gemini-3 Judges Gemini: {gemini_judges_gemini}
    Time: {gemini_judges_gemini_time:.3f}s

    GPT-5 Judges Gemini: {gpt_judges_gemini}
    Time: {gpt_judges_gemini_time:.3f}s

    Sonnet-4.5 Judges Sonnet: {sonnet_judges_sonnet}
    Time: {sonnet_judges_sonnet_time:.3f}s

    Gemini-3 Judges Sonnet: {gemini_judges_sonnet}
    Time: {gemini_judges_sonnet_time:.3f}s

    GPT-5 Judges Sonnet: {gpt_judges_sonnet}
    Time: {gpt_judges_sonnet_time:.3f}s
    """

async def run_fast_judge_loop():
    checkpoint_path_sonnet = '/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/fast_judge_results_sonnet.json'
    checkpoint_path_gemini = '/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/fast_judge_results_gemini.json'

    if os.path.exists(checkpoint_path_sonnet):
        with open(checkpoint_path_sonnet, 'r') as f:
            judge_results_sonnet = json.load(f)
        print(f"[Sonnet] Loaded checkpoint with {sum(len(v) for v in judge_results_sonnet.values())} entries")
    else:
        judge_results_sonnet = {}
        print("[Sonnet] Starting fresh")

    if os.path.exists(checkpoint_path_gemini):
        with open(checkpoint_path_gemini, 'r') as f:
            judge_results_gemini = json.load(f)
        print(f"[Gemini] Loaded checkpoint with {sum(len(v) for v in judge_results_gemini.values())} entries")
    else:
        judge_results_gemini = {}
        print("[Gemini] Starting fresh")

    base_path = '/content/drive/MyDrive/Medhavi AI/Physics QA Datasets/Diagram_Essential_Questions/Correct_Freq/'

    try:
        for folder_name in sorted(os.listdir(base_path)):
            chapter_path   = base_path + folder_name
            chapter_number = folder_name.split('_')[1]

            if chapter_number not in judge_results_sonnet: judge_results_sonnet[chapter_number] = {}
            if chapter_number not in judge_results_gemini: judge_results_gemini[chapter_number] = {}

            for question_number in sorted(os.listdir(chapter_path)):

                if (question_number in judge_results_sonnet[chapter_number] and
                    question_number in judge_results_gemini[chapter_number]):
                    print(f"Skipping Chap #{chapter_number} Q #{question_number} (already done)")
                    continue

                ans_path = chapter_path + '/' + question_number + '/a.png'
                print(f"Chap #{chapter_number} Q #{question_number}")

                try:
                    sonnet_response = claude[chapters_keys_gpt_and_claude[chapter_number]][question_number]
                    gemini_response = gemini_responses[chapters_keys_gemini[chapter_number]][question_number]

                    # All 6 calls fire simultaneously
                    result = await Even_Faster_LLM_Judge(sonnet_response, gemini_response, ans_path)
                    print(result)

                    judge_results_sonnet[chapter_number][question_number] = result
                    judge_results_gemini[chapter_number][question_number] = result

                    with open(checkpoint_path_sonnet, 'w') as f:
                        json.dump(judge_results_sonnet, f, indent=2)
                    with open(checkpoint_path_gemini, 'w') as f:
                        json.dump(judge_results_gemini, f, indent=2)

                except Exception as e:
                    print(f"ERROR Chap #{chapter_number} Q #{question_number}: {e}")
                    print(f"Restart from: chapter={chapter_number}, question={question_number}")

    except Exception as e:
        print(f"\nFATAL ERROR: {e}")
        with open(checkpoint_path_sonnet, 'w') as f: json.dump(judge_results_sonnet, f, indent=2)
        with open(checkpoint_path_gemini, 'w') as f: json.dump(judge_results_gemini, f, indent=2)
        raise

await run_fast_judge_loop()

[Sonnet] Starting fresh
[Gemini] Starting fresh
Chap #10 Q #115

    Sonnet-4.5 Judges Gemini: verdict: Correct, ground_truth: 1.23 rad/s, model_answer: 1.22 rad/s
    Time: 29.627s

    Gemini-3 Judges Gemini: verdict: Wrong, ground_truth: 1.23 rad/s, model_answer: 1.22 rad/s
    Time: 45.187s

    GPT-5 Judges Gemini: verdict: Correct, ground_truth: ω=1.23 rad/s, model_answer: ω=1.22 rad/s
    Time: 27.991s
    
    Sonnet-4.5 Judges Sonnet: verdict: Wrong, ground_truth: ω=1.23 rad/s, model_answer: ω=1.40 rad/s
    Time: 20.396s

    Gemini-3 Judges Sonnet: verdict: Wrong, ground_truth: 1.23 rad/s, model_answer: 1.40 rad/s
    Time: 20.132s

    GPT-5 Judges Sonnet: verdict: Wrong, ground_truth: ω=1.23 rad/s, model_answer: ω=1.40 rad/s
    Time: 19.680s
    
Chap #10 Q #123

    Sonnet-4.5 Judges Gemini: verdict: Correct, ground_truth: I=(2/5)mR², model_answer: I=(2/5)mR²
    Time: 9.417s

    Gemini-3 Judges Gemini: verdict: Correct, ground_truth: I = \frac{2}{5} m R^2, model_answer